In [ ]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

# loosely following this blog post: http://www.panoradio-sdr.de/correlation-for-time-delay-analysis/

In [ ]:
# import the binary file
rawData_benGS = np.fromfile(
    "/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/test5_benGS.dat",
    dtype="uint8",
)
rawData_domGS = np.fromfile(
    "/Users/benjaminpattison/Documents/Projects/satNav/SDR_satellite_tracking/data/test5_domGS.dat",
    dtype="uint8",
)

# subtract 255/2 to go from unsigned to signed values
benData = rawData_benGS - 127.5
domData = rawData_domGS - 127.5

# split the data into its real and imaginary parts
benRealParts = benData[0::2]
benImagParts = benData[1::2]
domRealParts = domData[0::2]
domImagParts = domData[1::2]

# create a variable that has the real and imaginary parts together
benCplx = benRealParts + 1j * benImagParts
domCplx = domRealParts + 1j * domImagParts


In [ ]:
# number of samples for each frequency
ben_nS = np.size(benCplx, 0)
dom_nS = np.size(domCplx, 0)
if ben_nS != dom_nS:
    print("You don't have the same number of samples from each ground station")
    exit()
else:
    nS_total = dom_nS
    nS_each = int(nS_total / 3)


In [ ]:
"""
Transmitter location according to wikipedia
https://geohack.toolforge.org/geohack.php?pagename=KUFX&params=37.205_N_121.950_W_type:landmark_region:US-CA_source:FCC
repeaters here
    37.151583    -121.609917 (SE of SJ)
    37.659389    -121.933028 (NE of Fremont)
"""
# correlation of the two signals for just the first reference sample. this method uses the standard complex correlation talked about here:
# http://www.panoradio-sdr.de/correlation-for-time-delay-analysis/
sampleRate = 2e6
xCorr = signal.correlate(benCplx[:nS_each], domCplx[:nS_each])
lags = signal.correlation_lags(len(benCplx[:nS_each]), len(domCplx[:nS_each]))


In [ ]:
# plot the correlation.
xVals = np.linspace(0, np.size(xCorr), np.size(xCorr))
plt.plot(np.abs(xCorr))
plt.show()
plt.plot(lags, np.abs(xCorr))
plt.show()
